# Titanic

## Can we leverage deep learning on irregular domains to save lifes?

---

*Teo Stocco, Pierre-Alexandre Lee, Yves Lamonato, Charles Thiebaut*, [EPFL](https://epfl.ch).

[Network Tour of Data Science](https://github.com/mdeff/ntds_2017) final project. This notebook contains a detailed overview through the whole project with all essential parts. As this work required several attempts and exploration, only relevant parts are kept here. You can however access their associated notebooks (unguided) with all processes when mentionned. This project was **not shared** with any other class.


[Binder access](https://mybinder.org/v2/gh/zifeo/Titanic/) | [nbviewer access](https://nbviewer.jupyter.org/github/zifeo/Titanic/blob/master/project.ipynb)

TODO:

- spell check plugin
- isolate code blocks in functions in separate Python module
- abstract
- notebook toc plugin
- README with getting started

## 1 Introduction

Icebergs and ships do not get well along each other. To avoid dramatic events such as the one that happened a century ago, we aim at helping a noble quest: differentiating icebergs and ships based on radar data to see whether any
iceberg is drifting away and might cross the road of a ship.

|© Statoil/C-CORE - Icebergs and ships examples|
|-|
|![](./img/statoil-ccore.png)|

This remote sensing measurements can be performed either by planes or by satellites. The latter can provide radar information up to 14 time a day as in the case of [Sentinel-1](https://fr.wikipedia.org/wiki/Sentinel-1). The C-Band radar manage to capture data in numerous conditions (e.g. darkness, rain, cloud, fog, etc.) and measures the energy reflected back called backscatter (Torres et al, 2012). Those data can latter be analyzed and used to clear out potential collision between icebergs and ships. 

Building on the top of recent advances in the field of signal processing on graphs (Schuman et al., 2013) and deep learning on irregular domains (Bronstein et al., 2017), we investigate the performance of standard machine learning methods and the relevance of graph based convolutional neural networks to perform binary classification in this specific case. The new method provide a convenient way of getting rotational invariance over the data and set up a flexible framework for structured pooling (Defferrard et al., 2017). Pooling operations require adequate aggregation by coarsening the graph between layers. We experiment how this framework can be exploited through various processes: Graclus multilevel algorithm, ... TODO

In [ ]:
% load_ext autoreload
% autoreload 2
% matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import torch
import pandas as pd
import sys

In [ ]:
# add gcnn module
sys.path.append('..')

# bigger figure
plt.rcParams['figure.figsize'] = 18, 5

In [ ]:
# setting seed for reproducability
np.random.seed(0)
torch.manual_seed(0);

## 2 Data source

The dataset is provided by Statoil, an oil and gas compagny, and C-CORE, a monitoring company using compute vision, to keep operations safe and efficient. It was released on Kaggle for prediction competition in late 2017. The full dataset contains 10'028 iceberg or ship cases with only 1'604 labelled. Some of the images were computer generated to avoid hand labelling in the competition. As we will only focus on labelled one, this should not matter. 

### Description

- HH: transmit and receive horizontally
- HV: transmit horizontally and receive vertically

Quantitative vs Qualitative


| Feature | Description | Type | Has N/A | Comment |
| - | - | - | - | - |
| id | | | | |
| band_1 | | | | unit?
| band_2 | | | | unit? |
| inc_angle | | | | what impact ? unit? cause? |
| is_iceberg | | | | |

In [ ]:
from gcnn.datasets import load_icebergs

measures = load_icebergs('train')
measures.head(4)

In [ ]:
measures.shape

### Exploration

> More details on associated notebook [xx]()

some more cases

Let's have a look at an example of iceberg.

In [ ]:
example_iceberg = measures.iloc[5]
example_iceberg

In [ ]:
from gcnn import viz

In [ ]:
viz.plot_bands(example_iceberg)

In [ ]:
viz.plot_bands_3d(example_iceberg)

- varies a lot in size
- "intensity" difference
- lots of backscatter noise

---

Let's now look at an example of a ship.

In [ ]:
example_ship = measures.iloc[0]
example_ship

In [ ]:
viz.plot_bands(example_ship)

In [ ]:
viz.plot_bands_3d(example_ship, angle=60)

- some elongated shape
- other angle but not visible from eye
- still lots of noise

---

What about the distribution of the data and correlation ?

In [ ]:
plt.title('iceberg distribution')
measures.groupby(measures.is_iceberg).is_iceberg.count().plot.barh();

distribution of bands and angle? covariance

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
pca = PCA(n_components=50).fit_transform(np.c_[np.stack(measures.band_1), np.stack(measures.band_2)])
# pca speed up T-SNE and suppress some noise
tsne = TSNE(n_components=2).fit_transform(pca)

In [ ]:
plt.scatter(tsne[measures.is_iceberg == 1, 0], tsne[measures.is_iceberg == 1, 1], label='icebergs')
plt.scatter(tsne[measures.is_iceberg == 0, 0], tsne[measures.is_iceberg == 0, 1], label='ships')
plt.legend();

- ?

### Prototypes

One interesting first point is to look whether there are some prototypes (distinct primitive shapes). This allows to gather insights that will be later useful if there is any unbalanced between icebergs and ships. For example, one could try to cluster similar band 1 together:

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
n_proto = 4
kmeans = KMeans(n_clusters=n_proto).fit(np.stack(measures.band_1))
kmeans_centers = kmeans.cluster_centers_

In [ ]:
for i, center in enumerate(kmeans_centers):
    plt.subplot(1, 4, i % 4 + 1)
    plt.imshow(center.reshape(75, 75))
    if i % 4 == 3:
        plt.show()

In [ ]:
measures.is_iceberg.groupby(kmeans.labels_).apply(pd.value_counts)

- Scatter effect

---

Looking at a graph based method, ...

## 3 Preprocessing

jusitfiy "no feature" extraction

### Train, validation and test splits

later crossval?

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, valtest = train_test_split(range(len(measures)), test_size=0.3, stratify=measures.is_iceberg)
validation, test = train_test_split(valtest, test_size=0.5, stratify=measures.iloc[valtest].is_iceberg)

In [ ]:
measures.iloc[train].is_iceberg.value_counts()

In [ ]:
measures.iloc[validation].is_iceberg.value_counts()

In [ ]:
measures.iloc[test].is_iceberg.value_counts()

### Cleaning and scaling

only angle, among the trials we saw that replace it by median/mean/fixed number

justify scaling assumption

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
band_scaler = MinMaxScaler()
angle_scaler = MinMaxScaler()

In [ ]:
for _, e in measures.iloc[train].iterrows():
    band_scaler.partial_fit(e.band_1.reshape(1, -1))
    band_scaler.partial_fit(e.band_2.reshape(1, -1))
    
angle_scaler.fit(measures.iloc[train].inc_angle.dropna().values.reshape(-1, 1));

In [ ]:
# justify
angle_na_replacement = measures.iloc[train].inc_angle.dropna().mean()
angle_na_replacement

### Usable data

In [ ]:
bands = np.stack([
    band_scaler.transform(np.stack(measures.band_1)),
    band_scaler.transform(np.stack(measures.band_2)),
], axis=1).reshape(-1, 2, 75, 75)

In [ ]:
angles = angle_scaler.transform(measures.inc_angle.fillna(angle_na_replacement).values.reshape(1, -1))

In [ ]:
targets = measures.is_iceberg

## 4 Graphs

### Grid

circular

In [ ]:
from gcnn import graph

In [ ]:
small_grid = nx.grid_graph([4, 4, 1])
nx.draw(small_grid)

In [ ]:
small_knn = graph.knn(graph.grid_coordinates(5), k=8, metric='cityblock')
small_knn = nx.from_numpy_array(small_knn)
nx.draw(small_knn)

In [ ]:
small_wraps = graph.kwraps(5, k=1)
small_wraps = nx.from_numpy_matrix(small_wraps)        
nx.draw(small_wraps)

In [ ]:
small_grid3d = nx.grid_graph([4, 4, 2])
nx.draw(small_grid3d)

In [ ]:
small_knn3d = graph.knn(
    np.r_[
        np.c_[graph.grid_coordinates(5), np.zeros(5 * 5)],
        np.c_[graph.grid_coordinates(5), np.ones(5 * 5)],
    ],
    k=16, 
    metric='cityblock'
)
small_knn3d = nx.from_numpy_array(small_knn3d)
nx.draw(small_knn3d)

In [ ]:
small_wraps3d = graph.kwraps3d(5, k=1, d=2)
nx.draw(small_wraps3d)

analyze those graph in terms of connectivity etc as in homework 2 using networkx power

In [ ]:
dict(small_wraps3d.degree).values()

### Pattern-based graph

## 5 Models

In [ ]:
scores = pd.DataFrame(dict(name=[], accuracy=[], accuracy_std=[])).set_index('name')

scores.loc['baseline_random'] = [1 - measures.is_iceberg.sum() / len(measures), 0]
scores

### Naive methods

knn, logistic

In [ ]:
flat_features = np.c_[bands.reshape(-1, 2 * 75 * 75), angles.reshape(-1, 1)]
flat_features.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2).fit(flat_features[train], targets[train])
knn_predict = knn.predict(flat_features[test])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(targets[test], knn_predict), target_names=['ship', 'iceberg'])

### Graph Convolution

fourier

chebyshev

### Pooling

graclus implement is courtesy of Michael Defferrard

korn?

algeibraic (spectral cut)

MST

pattern-based

## 6 Training


### Skorch

## 7 Evaluation

### Models

### Stacking

## 8 Conclusion

### Futures work

### Acknowledgements

## 9 References

- TORRES, Ramon, SNOEIJ, Paul, GEUDTNER, Dirk, et al. GMES Sentinel-1 mission. Remote Sensing of Environment, 2012, vol. 120, p. 9-24.
- SHUMAN, David I., NARANG, Sunil K., FROSSARD, Pascal, et al. The emerging field of signal processing on graphs: Extending high-dimensional data analysis to networks and other irregular domains. IEEE Signal Processing Magazine, 2013, vol. 30, no 3, p. 83-98.
- BRONSTEIN, Michael M., BRUNA, Joan, LECUN, Yann, et al. Geometric deep learning: going beyond euclidean data. IEEE Signal Processing Magazine, 2017, vol. 34, no 4, p. 18-42.
- DEFFERRARD, Michaël, BRESSON, Xavier, et VANDERGHEYNST, Pierre. Convolutional neural networks on graphs with fast localized spectral filtering. In : Advances in Neural Information Processing Systems. 2016. p. 3844-3852.